<a href="https://colab.research.google.com/github/MathMachado/Python_RFB/blob/master/Projetos/Titanic/3DP_Outliers%20Handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TITANIC
## 3DP - DATA PREPARATION
## 3DP - OUTLIERS HANDLING

# Machine Learning com Python (Scikit-Learn)

![Scikit-Learn](https://github.com/MathMachado/Python_RFB/blob/master/Material/scikit-learn-1.png?raw=true)

# Carregar as bibliotecas (genéricas) Python

In [0]:
# Pandas
import pandas as pd
from pandas import Series, DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

# Carregar Dados

In [0]:
url= 'https://raw.githubusercontent.com/MathMachado/Python_RFB/master/Dataframes/df_3DP_MVH.csv?token=AGDJQ63OG3WQQJH4RLXXU4K5NWBPK'

# Carrega os dataframes de treinamento e teste e define 'PassengerId' como chave
df= pd.read_csv(url, index_col='PassengerId')
df.head(50)

# Boxplot

![BoxPlot](https://github.com/MathMachado/Python_RFB/blob/master/Material/boxplot.png?raw=true)

In [0]:
def PlotaBoxPlot_Survived(df, column):
    plt.rcdefaults()
    # make boxplot with Catplot
    sns.catplot(x='Survived2', y= column, kind="box",  data=df, height=4,aspect=1.5)
    # add data points to boxplot with stripplot
    sns.stripplot(x='Survived2', y= column, data=df, alpha=0.3,jitter=0.2,color='k');
    plt.show()

## Variável 'Age2'

In [0]:
PlotaBoxPlot_Survived(df, 'Age2')

Qual a conclusão?

## Family_Size

In [0]:
PlotaBoxPlot_Survived(df, 'Family_Size')

Qual a conclusão?

## Fare

In [0]:
PlotaBoxPlot_Survived(df, 'Fare')

Qual a conclusão?

# Z-Score

* Z-Score pode ser utilizado para detectar Outliers.
* É a diferença entre o valor e a média da amostra expressa como o número de desvios-padrão. 
* Se o escore z for menor que 2,5 ou maior que 2,5, o valor estará nos 5% do menor ou maior valor (2,5% dos valores em ambas as extremidades da distribuição). No entanto, é pratica comum utilizarmos 3 ao invés dos 2,5.

![Z_Score](https://github.com/MathMachado/Python_RFB/blob/master/Material/Z_Score.png?raw=true)

Abaixo, definimos a função para detectar os outliers baseados no Z-Score:

In [0]:
from scipy.stats import zscore
def ZScore_Outlier_Detect(column):
    df[column+'_ZS'] = zscore(df[column])
    df[column+'__is_outlier'] = df[column+'_ZS'].apply(lambda x: x <= -3 or x >= 3)
    df_AUX= df[df[column+'__is_outlier']== False]
    min_vlr= df_AUX[column].min()
    max_vlr= df_AUX[column].max()    
    df[column+'_Outlier_ZS']= df[column]
    
    df.loc[df[column+'_Outlier_ZS'] < min_vlr, column+'_Outlier_ZS'] = min_vlr
    df.loc[df[column+'_Outlier_ZS'] > max_vlr, column+'_Outlier_ZS'] = max_vlr
 
    df.drop(columns= [column+'_ZS', column+'__is_outlier'], axis=1, inplace= True)

# IQR Score

* O Intervalo interquartil (IQR) é uma medida de dispersão estatística, sendo igual à diferença entre os percentis 75 e 25, ou entre quartis superiores e inferiores, IQR = Q3 - Q1.

![BoxPlot](https://github.com/MathMachado/Python_RFB/blob/master/Material/boxplot.png?raw=true)

Abaixo, a função para detectar outliers baseados no IQR Score:

In [0]:
def IQR_Score_Outlier_Detect(column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    Lim_Inf= Q1-1.5*IQR
    Lim_Sup= Q3+1.5*IQR
    df[column+'__is_outlier'] = df[column].apply(lambda x: x <= Lim_Inf or x >= Lim_Sup)
    
    df_AUX= df[df[column+'__is_outlier']== False]
    min_vlr= df_AUX[column].min()
    max_vlr= df_AUX[column].max()    
    df[column+'_Outlier_IQR']= df[column]
    
    df.loc[df[column+'_Outlier_IQR'] < min_vlr, column+'_Outlier_IQR'] = min_vlr
    df.loc[df[column+'_Outlier_IQR'] > max_vlr, column+'_Outlier_IQR'] = max_vlr
 
    df.drop(columns= [column+'__is_outlier'], axis=1, inplace= True)

# Avaliando Outliers: variáveis 'Age2', 'Age3', 'Age_Inf' e 'Fare'

In [0]:
#df= df_copia7.copy()
df_copia7= df.copy()
df.head()

In [0]:
df_Outlier= df[['Age2', 'Age3', 'Fare']].copy()
df_Outlier.head()

A seguir, vamos detectar outliers usando os dois métodos e depois decidimos qual variável tem melhor poder preditivo.

In [0]:
ZScore_Outlier_Detect('Age2')
ZScore_Outlier_Detect('Age3')
ZScore_Outlier_Detect('Fare')

In [0]:
IQR_Score_Outlier_Detect('Age2')
IQR_Score_Outlier_Detect('Age3')
IQR_Score_Outlier_Detect('Fare')

Vamos tabular alguns resultados para avaliar o impacto destas transformações na variável-resposta 'Survived':

In [0]:
print(round(df[['Age2', 'Age2_Outlier_ZS', 'Age2_Outlier_IQR', 'Survived2']].groupby('Survived2').mean(),0))

Qual sua opinião com relação à estes resultados para a variável 'Age'?

In [0]:
print(round(df[['Age3','Age3_Outlier_IQR','Age3_Outlier_ZS','Survived2']].groupby('Survived2').mean(),0))

Qual sua opinião com relação à estes resultados?

In [0]:
print(round(df[['Fare', 'Fare_Outlier_ZS', 'Fare_Outlier_IQR', 'Survived2']].groupby('Survived2').mean(),0))

Qual sua opinião com relação à estes resultados?

# Conclusão

In [0]:
df.head(10)

# Salvar cópia do dataframe tratado nesta fase

In [0]:
df.to_csv("df_3DP_OH.csv", sep= ',', index = True, header=True)